In [1]:
import numpy as np
import pandas as pd

In [2]:
#scores = pd.read_csv('ml-20m/genome-scores.csv')
#g_tags = pd.read_csv('ml-20m/genome-tags.csv')
#links = pd.read_csv('ml-20m/links.csv')
#movies = pd.read_csv('ml-20m/movies.csv')
ratings = pd.read_csv('ml-20m/ratings.csv')
#tags = pd.read_csv('ml-20m/tags.csv')

In [14]:
import pymongo

In [15]:
from pymongo import MongoClient
client = MongoClient()

In [16]:
db1 = client.imdb_movies

In [54]:
links.drop('tmdbId', inplace=True, axis=1)

In [45]:
import urllib
import bs4
import requests
from bs4 import BeautifulSoup

In [146]:
base_movie_url = "https://www.imdb.com/title/tt0"
#page = requests.get(base_movie_url + str(links['imdbId'].iloc[1]) + '/')
#page.status_code
base_movie_url2 = "https://www.imdb.com/title/tt00"

In [14]:
#soup1 = BeautifulSoup(page.content, 'html.parser')

In [173]:
def create_movie_data(movie_id):
    if int(movie_id) // 100000 == 0 and int(movie_id) // 10000 > 0:
        page = requests.get(base_movie_url2 + movie_id + '/')
    elif int(movie_id) // 1000000 == 0 and int(movie_id) // 100000 > 0:
        page = requests.get(base_movie_url + movie_id + '/')
    if page.status_code == 200:
        print("Received successful page response")
        soup = BeautifulSoup(page.content, 'html.parser')
        movie = {}
        movie[str(movie_id)] = {}
        movie[str(movie_id)]['title'], movie[str(movie_id)]['release'] = get_title_and_release(soup)
        movie[str(movie_id)]['thumbnail_addr'] = get_thumbnail_addr(soup)
    elif page.status_code == 404:
        movie = {}
        print("Received No Data")
    return movie
        

In [48]:
def get_title_and_release(soup):
    title_year = soup.title.text
    title_year = title_year[:-8]
    year = title_year[-4:]
    title = title_year[:-6]
    return title, year

In [49]:
def get_thumbnail_addr(soup):
    scripts = soup.findAll('script')
    for i, element in enumerate(scripts):
        if "application/ld+json" in str(element):
            json = element
    json = str(json).split('\n')
    for i, element in enumerate(json):
        if "\"image\"" in element:
            image = element
    image = image.split()[-1][1:-2]
    return image

In [11]:
genres = ['Adventure']
for i in range(len(movies['genres'].iloc[:])):
    temp_movie_genre = movies['genres'].iloc[i]
    temp_movie_genre = temp_movie_genre.split('|')
    for i, element in enumerate(temp_movie_genre):
        if element not in genres:
            genres.append(element)

In [ ]:
#Getting 30 movies for each genre
movies.head()
def get_genres(string):
    return string.split('|')

#get_genres(movies.iloc[0,2])

movie_ids = []

genre_count = dict(zip(genres, np.zeros(len(genres))))

flag = 0
for i in range(len(movies.iloc[:, 0])):
    flag = 0
    temp_movie = movies['title'].iloc[i]
    temp_genres = get_genres(movies['genres'].iloc[i])
    for j, element in enumerate(temp_genres):
        print(element)
        if genre_count[element] >= 15:
            print('Flag is ', 1)
            flag = 1
        print('Flag is ', 0)
    if flag == 1:
        continue
    else:
        for j, element in enumerate(temp_genres):
            genre_count[element] += 1
        movie_ids.append(movies['movieId'].iloc[i])
        print("New Movie added")
    

In [209]:
def get_movies(movie_ids):
    """
    Generates final movies given genres, movies dataframe and number_of_movies_for_each_genre
    """
    movies = {}
    for i, element in enumerate(movie_ids):
        if i<86 or i in [123,172,175,178,180,181]:
            continue
        imdbId = str(links[links['movieId'] == int(element)]['imdbId'].iloc[0])
        movie_data = create_movie_data(imdbId)
        if len(movie_data[imdbId].items()) != 3:
            print("Error at ", i)
            continue
        movies[str(element)] = movie_data[imdbId]
        print(i + 1, " done")
    return movies

In [ ]:
to_post1 = get_movies(movie_ids)

In [4]:
db = client.imdb_movies

In [6]:
post = db.post
#post.insert_many(list(to_post.values()))

In [12]:
#db.collection_names(include_system_collections=False)
import pprint
pprint.pprint(post.find_one())

{'_id': ObjectId('5c634a5ce3c0160aef1e93fd'),
 'release': '1995',
 'thumbnail_addr': 'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_.jpg',
 'title': 'Toy Story'}


In [ ]:
for temp in post.find():
    pprint.pprint(temp)

In [6]:
user_ratings = {}
for k, v in ratings.groupby('userId'):
    user_ratings[k] = dict(zip(v['movieId'].values, v['rating'].values))

In [18]:
users = db1['users']

In [19]:
i = 0
for k, v in user_ratings.items():
    to_post = {}
    to_post['_id'] = k
    for k2, v2 in v.items():
        to_post[str(k2)] = v2
    users.insert_one(to_post)

In [81]:
import pprint
for doc in users.find():
    pprint.pprint(doc)
    if 1:
        break
#import pickle
#with open('to_post', 'rb') as file:
#    movie_data = pickle.load(file)

{'1': 5.0,
 '100': 5.0,
 '1012': 4.0,
 '1028': 4.0,
 '1030': 5.0,
 '1031': 4.0,
 '1035': 4.0,
 '1036': 5.0,
 '1038': 2.0,
 '104': 5.0,
 '1042': 4.0,
 '105': 2.0,
 '1073': 5.0,
 '1084': 5.0,
 '1088': 4.0,
 '1091': 4.0,
 '1092': 5.0,
 '1093': 3.0,
 '1095': 5.0,
 '11': 5.0,
 '1100': 4.0,
 '1101': 5.0,
 '1103': 5.0,
 '111': 5.0,
 '1120': 4.0,
 '1129': 4.0,
 '1135': 4.0,
 '1179': 5.0,
 '1183': 1.0,
 '1193': 5.0,
 '1197': 4.0,
 '1201': 5.0,
 '1206': 5.0,
 '1207': 4.0,
 '1213': 5.0,
 '1219': 5.0,
 '122': 4.0,
 '1220': 5.0,
 '1221': 4.0,
 '1222': 5.0,
 '1228': 4.0,
 '1234': 5.0,
 '1240': 4.0,
 '1247': 5.0,
 '1252': 5.0,
 '1256': 5.0,
 '1265': 5.0,
 '1267': 4.0,
 '1271': 5.0,
 '1278': 5.0,
 '1287': 4.0,
 '1290': 5.0,
 '1302': 5.0,
 '1304': 4.0,
 '1307': 5.0,
 '1333': 5.0,
 '1334': 4.0,
 '1340': 5.0,
 '1359': 3.0,
 '1370': 4.0,
 '1377': 4.0,
 '1380': 3.0,
 '1387': 4.0,
 '1393': 5.0,
 '14': 3.0,
 '1401': 3.0,
 '1431': 1.0,
 '1439': 5.0,
 '1441': 3.0,
 '1461': 3.0,
 '1466': 5.0,
 '1476': 5.0,
 '14

In [27]:
#pprint.pprint(users.find_one({'_id':1}))


In [34]:
movie_ids = list(movie_data.keys())

In [45]:
max_count = {}
for k, v in user_ratings.items():
    count = 0
    #to_post = {}
    #to_post['_id'] = k
    for k2, v2 in v.items():
        if str(k2) in movie_ids:
            count+=1
        #to_post[str(k2)] = v2
    max_count[str(k)] = count

In [52]:
sorted_count = sorted(max_count.items(), key = lambda x:x[1], reverse=True)

In [79]:
to_post_users = dict(sorted_count[15000:15100])

In [80]:
for k, v in user_ratings.items():
    if str(k) in list(to_post_users.keys()):
        to_post = {}
        to_post['_id'] = k
        for k2, v2 in v.items():
            to_post[str(k2)] = v2
        users.insert_one(to_post)